## EasyCV图像自监督训练-MAE
本文将介绍如何利用EasyCV使用自监督算法[MAE](https://arxiv.org/pdf/2111.06377.pdf)进行图像自监督模型的训练
 


## 运行环境要求

PAI-Pytorch镜像 or 原生Pytorch1.5+以上环境 GPU机器， 内存32G以上

## 安装依赖包

注: 在PAI-DSW docker中无需安装相关依赖，可跳过此部分 在本地notebook环境中执行


1、 首先，安装pytorch和对应版本的torchvision，支持Pytorch1.5.1以上版本

In [ ]:
# install pytorch and torch vision
! conda install --yes pytorch==1.10.0 torchvision==0.11.0 -c pytorch

2、获取torch和cuda版本，安装对应版本的mmcv和nvidia-dali

In [ ]:
import torch
import os
os.environ['CUDA']='cu' + torch.version.cuda.replace('.', '')
os.environ['Torch']='torch'+torch.version.__version__.replace('+PAI', '')
!echo "cuda version: $CUDA"
!echo "pytorch version: $Torch"

In [ ]:
# install some python deps
! pip install mmcv-full==1.4.4 -f https://download.openmmlab.com/mmcv/dist/${CUDA}/${Torch}/index.html
! pip install http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/third_party/nvidia_dali_cuda100-0.25.0-1535750-py3-none-manylinux2014_x86_64.whl

3、  安装EasyCV算法包

In [ ]:
pip install pai-easycv

4、 简单验证

In [ ]:
from easycv.apis import *

## 数据准备

自监督训练只需要提供无标注图片即可进行， 你可以下载[ImageNet](http://www.image-net.org/download-images) 数据，或者使用你自己的图片数据。需要提供一个包含若干图片的文件夹路径`p`，以及一个文件列表，文件列表中是每个图片相对图片目录`p`的路径

图片文件夹结构示例如下, 文件夹路径为`./images`

```shell
images/
├── 0001.jpg
├── 0002.jpg
├── 0003.jpg
|...
└── 9999.jpg
```

文件列表内容示例如下
```text
0001.jpg
0002.jpg
0003.jpg
...
9999.jpg
```

为了快速走通流程，我们也提供了一个小的示例数据集，执行如下命令下载解压

In [ ]:
! wget http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/data/imagenet_raw_demo/imagenet_raw_demo.tar.gz && tar -zxf imagenet_raw_demo.tar.gz

In [ ]:
# 重命名文件夹
! mv imagenet_raw_demo  imagenet_raw

## 模型训练

这个Demo中我们采用[MAE](https://arxiv.org/pdf/2111.06377.pdf)自监督算法训练vit-base主干网络， 下载示例配置文件

In [ ]:
! rm -rf mae_vit_base_patch16_8xb64_1600e.py
! wget http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/release/doc/easycv/configs/selfsup/mae/mae_vit_base_patch16_8xb64_1600e.py

为了缩短训练时间，打开配置文件 `mae_vit_base_patch16_8xb64_1600e.py`，修改`total_epoch`参数为5， 每隔1次迭代打印一次日志。

```python
# runtime settings
total_epochs = 5

# log config
log_config=dict(interval=1)
```

正式训练时，建议使用`多机8卡`或`单机8卡`配合该配置文件使用，修改update_interval参数进行梯度累积，确保有效batch_size一致

In [ ]:
# 查看easycv安装位置
import easycv
print(easycv.__file__)

In [ ]:
!python -m torch.distributed.launch --nproc_per_node=1 --master_port=29930 \
/home/pai/lib/python3.6/site-packages/easycv/tools/train.py mae_vit_base_patch16_8xb64_1600e.py --work_dir work_dir/selfsup/jpg/mae --launcher pytorch

## 模型导出
对模型的字段进行修改,以便用于fintune任务

In [ ]:
import torch 
weight_path = 'work_dir/selfsup/jpg/mae/epoch_5.pth'
state_dict = torch.load(weight_path)['state_dict']
state_dict_out = {}
for key in state_dict:
    state_dict_out[key.replace('encoder.','')] = state_dict[key]
torch.save(state_dict_out,weight_path)

## 使用自监督模型进行图像分类fintune
下载分类任务示例配置文件

In [ ]:
! rm -rf mae_vit_base_patch16_8xb64_100e_lrdecay065_fintune.py
! wget http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/release/doc/easycv/configs/selfsup/mae/mae_vit_base_patch16_8xb64_100e_lrdecay065_fintune.py

修改配置文件 `mae_vit_base_patch16_8xb64_1600e.py`，修改`model.pretrained`参数为`work_dir/selfsup/jpg/mae/epoch_5.pth`，为了缩短训练时间，修改`total_epoch`参数为5，每隔1次迭代打印一次日志。

```python
# runtime settings
total_epochs = 5

# log config
log_config=dict(interval=1)

# 
pretrained='work_dir/selfsup/jpg/mae/epoch_5.pth'
```

正式训练时，建议使用`多机8卡`或`单机8卡`配合该配置文件使用，修改update_interval参数进行梯度累积，确保有效batch_size一致

### 分类模型训练
这里提供了单卡进行训练和验证集评估的命令

In [ ]:
!python -m torch.distributed.launch --nproc_per_node=1 --master_port=29930 \
/home/pai/lib/python3.6/site-packages/easycv/tools/train.py mae_vit_base_patch16_8xb64_100e_lrdecay065_fintune.py --work_dir work_dir/selfsup/jpg/mae --launcher pytorch

### 预测
参考EasyCV图像分类的demo，对训练好的模型导出并预测